In [1]:
import math
from datetime import datetime, date, timedelta
import random
import matplotlib.pyplot as plt
from statistics import mean
import imp
import os
import numpy as np
from pulp import *
import warnings
from scipy.special import softmax

from haolun_algorithm.offline_LP import offline_LP
from haolun_algorithm.online_ADAP import online_ADAP
from haolun_algorithm.online_Greedy import online_Greedy

warnings.filterwarnings('ignore')

/var/folders/0t/55spm7k95h33phsgfwny8m600000gn/T/ipykernel_89523/2354497342.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
"""
Simulate all the needed information
# LHS (U): server nodes, RHS (V): call type nodes, W: weights
# pvt: probability of v coming at time t
# T: time horizons, K: occupied time steps
# lambd: parameter for exponential distribution
"""
# Create the 'prob' variable to contain the problem data
prob = LpProblem("matching", LpMaximize)
# test the offline optimal
U, V = 6, 4
T, K = 5, 2
lambd = 0.5
W = np.array([[[random.uniform(0, 1) for j in range(V)] for i in range(U)] for t in range(T)])
pvt = [[1. / V for v in range(V)] for t in range(T)]
LHS, RHS = range(U), range(V)

In [3]:
"""
Offline optimal
"""
Xopt, optimal = offline_LP(prob, LHS, RHS, W, pvt, T, K, lambd)
print("offline weight:", optimal)

(5, 6, 4)
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/haolunwu/opt/anaconda3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/0t/55spm7k95h33phsgfwny8m600000gn/T/f9b6061da67348ab845d69667a3f16ae-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/0t/55spm7k95h33phsgfwny8m600000gn/T/f9b6061da67348ab845d69667a3f16ae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 55 COLUMNS
At line 656 RHS
At line 707 BOUNDS
At line 828 ENDATA
Problem MODEL has 50 rows, 120 columns and 480 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 50 (0) rows, 120 (0) columns and 480 (0) elements
0  Obj -0 Dual inf 59.665944 (120)
34  Obj 7.0644089 Primal inf 14.374998 (19)
61  Obj 4.3432703
Optimal - objective value 4.3432703
Optimal objective 4.343270328 - 61 iterations time 0.002
Option for printingOptions c

In [4]:
Xopt

[[[0.0, 0.0, 0.25, 0.25],
  [0.25, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.25, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.25, 0.25],
  [0.25, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.25, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.25, 0.0],
  [0.25, 0.25, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.25],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.25, 0.25, 0.0, 0.0],
  [0.0, 0.0, 0.25, 0.25],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.25],
  [0.25, 0.25, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.25, 0.0]]]

In [5]:
"""Check constraint"""
print("constraint 1: no customer over matched")
for t in range(T):
    for v in RHS:
        print(np.sum(Xopt[t][u][v] for u in LHS) <= 1)
        
print("constraint 2: no server over matched")
for t in range(T):
    for u in LHS:
        print(np.sum(
                Xopt[tau][u][v] * math.exp(-lambd * (t - tau)) for v in RHS for tau in range(t - K, t)) + np.sum(
                Xopt[t][u][v] for v in RHS) <= 1)
#         print(np.sum(Xopt[tau][u][v] for v in RHS for tau in range(t - K, t)) 
#               + np.sum(Xopt[t][u][v] for v in RHS) <= 1)

constraint 1: no customer over matched
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
constraint 2: no server over matched
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
def sampleArrival(pv, RHS):
    r = random.uniform(0, 1)
    cur_sum = 0
    for cur_v in RHS:
        cur_sum += pv[cur_v]
        if r <= cur_sum:
            return cur_v
    return len(RHS) - 1


simulate_cur_v = []
for t in range(T):
    p_v = pvt[t]
    simulate_cur_v.append(sampleArrival(p_v, RHS))


for i in range(T):
    print("W_curr:", W[i][:,simulate_cur_v[i]])

W_curr: [0.49024321 0.97549946 0.91603154 0.90569069 0.31643436 0.03760273]
W_curr: [0.29074924 0.25520005 0.60648162 0.86325801 0.12422612 0.49475292]
W_curr: [0.11625099 0.289233   0.49870788 0.7956885  0.87044615 0.8635539 ]
W_curr: [0.27176888 0.90012678 0.22877167 0.21786354 0.20407346 0.35344895]
W_curr: [0.37445059 0.73983923 0.62399429 0.36832302 0.00471047 0.19503592]


In [8]:
"""
# Greedy
# """
run=1
weight_avg_greedy = 0
for _ in range(run):
    matched_pairs, online_Greedy_weight = online_Greedy(LHS, RHS, W, pvt, T, K, Xopt)
    weight_avg_greedy += online_Greedy_weight
print("weight_avg_greedy:", weight_avg_greedy / run)

weight_avg_greedy: 4.215833865170753


In [9]:
matched_pairs

{0: (1, 2), 1: (3, 1), 2: (4, 3), 3: (0, 1), 4: (5, 2)}

In [10]:
greedy_x = [[[0 for j in range(V)] for i in range(U)] for t in range(T)]
for t in matched_pairs:
    (i, j) = matched_pairs[t]
    greedy_x[t][i][j]=1
greedy_x

[[[0, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]],
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]],
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 0, 0, 0]],
 [[0, 1, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]],
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 1, 0]]]

In [11]:
res_greedy = 0
for t in range(T):
    for u in LHS:
        for v in RHS:
            res_greedy += W[t][u][v] * greedy_x[t][u][v]
res_greedy
            

4.215833865170753

In [12]:
"""Check constraint"""
print("constraint 1: no customer over matched")
for t in range(T):
    for v in RHS:
        print(np.sum(greedy_x[t][u][v] for u in LHS) <= pvt[t][v])
        
print("constraint 2: no server over matched")
for t in range(T):
    for u in LHS:
        # prob += np.sum(
        #     X[tau][u][v] * math.exp(-lambd * (t - tau)) for v in RHS for tau in range(t - K, t)) + np.sum(
        #     X[t][u][v] for v in RHS) <= 1
        print(np.sum(greedy_x[tau][u][v] for v in RHS for tau in range(t - K, t)) 
              + np.sum(greedy_x[t][u][v] for v in RHS) <= 1)

constraint 1: no customer over matched
True
True
False
True
True
False
True
True
True
True
True
False
True
False
True
True
True
True
False
True
constraint 2: no server over matched
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
